In [55]:
# Batch config example
print("Example batch config JSON for upload:")
print("""
[
    {
        "patient": "patient_id1",
        "data_type": "cns",
        "archives": ["archive1"],
        "measurements": ["EEG1"],
        "start_time": "2023-01-01T00:00:00+00:00",
        "end_time": "2023-01-01T12:00:00+00:00",
        "processing": ["Median Filter"],
        "export_format": "CSV",
        "median_window_size": 5
    },
    {
        "patient": "patient_id2",
        "data_type": "edf",
        "archives": ["file1.edf"],
        "measurements": ["F3-C3"],
        "start_time": "2023-01-02T00:00:00+00:00",
        "end_time": "2023-01-02T06:00:00+00:00",
        "processing": ["None"],
        "export_format": "EDF",
        "median_window_size": 3
    }
]
""")

Example batch config JSON for upload:

[
    {
        "patient": "patient_id1",
        "data_type": "cns",
        "archives": ["archive1"],
        "measurements": ["EEG1"],
        "start_time": "2023-01-01T00:00:00+00:00",
        "end_time": "2023-01-01T12:00:00+00:00",
        "processing": ["Median Filter"],
        "export_format": "CSV",
        "median_window_size": 5
    },
    {
        "patient": "patient_id2",
        "data_type": "edf",
        "archives": ["file1.edf"],
        "measurements": ["F3-C3"],
        "start_time": "2023-01-02T00:00:00+00:00",
        "end_time": "2023-01-02T06:00:00+00:00",
        "processing": ["None"],
        "export_format": "EDF",
        "median_window_size": 3
    }
]



In [62]:
import os
import glob
import pandas as pd
import numpy as np
import h5py
import pyedflib
import mne
from cns_utils.readCNS import CNSDataSource
from scipy.signal import butter, filtfilt, medfilt
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, FileLink, HTML
import json
import uuid
import traceback
import pytz
import time
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
import pkg_resources
import psutil

# Diagnostic check for ipywidgets
def check_widgets_environment():
    try:
        ipywidgets_version = pkg_resources.get_distribution("ipywidgets").version
        print(f"ipywidgets version: {ipywidgets_version}")
        with status_output:
            print(f"ipywidgets version: {ipywidgets_version}")
        try:
            import jupyter_nbextension
            print("widgetsnbextension is installed.")
            with status_output:
                print("widgetsnbextension is installed.")
        except ImportError:
            print("widgetsnbextension is not installed. Run: jupyter nbextension enable --py widgetsnbextension --sys-prefix")
            with status_output:
                print("widgetsnbextension is not installed. Run: jupyter nbextension enable --py widgetsnbextension --sys-prefix")
    except Exception as e:
        print(f"Error checking ipywidgets environment: {str(e)}")
        with status_output:
            print(f"Error checking ipywidgets environment: {str(e)}")

# Configuration and environment setup
WORKSPACE = os.getenv('WORKSPACE', 'tracktbi').lower()
BASE_PATH = f'/mnt/s3/{WORKSPACE}-data-main'
EXPORT_DIR = 'exported_data'
EXPORT_SUMMARIES_DIR = 'export_summaries'
CONFIG_DIR = 'configs'
os.makedirs(EXPORT_DIR, exist_ok=True)
os.makedirs(EXPORT_SUMMARIES_DIR, exist_ok=True)
os.makedirs(CONFIG_DIR, exist_ok=True)
EXPORT_TIMEOUT = 1800  # 30 minutes per measurement

# Signal processing functions
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return filtfilt(b, a, data)

def median_filter(data, kernel_size=3):
    if kernel_size % 2 == 0 or kernel_size < 1:
        raise ValueError("Median filter kernel size must be a positive odd integer")
    return medfilt(data, kernel_size=kernel_size)

# Data reading functions
def get_patient_ids():
    try:
        return [os.path.basename(p) for p in glob.glob(os.path.join(BASE_PATH, '*')) if os.path.isdir(p)]
    except Exception as e:
        with status_output:
            print(f"Error: Failed to list patients in {BASE_PATH}. Ensure access to directory. Details: {str(e)}")
            traceback.print_exc()
        return []

def get_data_types(patient_id):
    try:
        patient_path = os.path.join(BASE_PATH, patient_id)
        return ['none'] + [os.path.basename(p) for p in glob.glob(os.path.join(patient_path, '*')) if os.path.isdir(p)]
    except Exception as e:
        with status_output:
            print(f"Error: Failed to list data types for patient {patient_id}. Check if patient exists. Details: {str(e)}")
            traceback.print_exc()
        return ['none']

def get_patient_archives(patient_id, data_type):
    try:
        if data_type == 'none':
            return []
        data_path = os.path.join(BASE_PATH, patient_id, data_type)
        if data_type == 'cns':
            return [os.path.basename(p) for p in glob.glob(os.path.join(data_path, '*')) if os.path.isdir(p)]
        elif data_type == 'edf':
            return [os.path.basename(p) for p in glob.glob(os.path.join(data_path, '*.edf'))]
        return []
    except Exception as e:
        with status_output:
            print(f"Error: Failed to list archives for patient {patient_id}, data type {data_type}. Details: {str(e)}")
            traceback.print_exc()
        return []

def get_cns_measurements(cns_dir):
    try:
        data_source = CNSDataSource(cns_dir)
        return data_source.get_basenames()
    except Exception as e:
        with status_output:
            print(f"Error: Failed to list measurements in {cns_dir}. Ensure CNS data is accessible. Details: {str(e)}")
            traceback.print_exc()
        return []

def get_time_range(patient_id, data_type, archives, measurements):
    if not archives or not measurements:
        with status_output:
            print("Error: No archives or measurements selected. Please select one archive and at least one measurement.")
        return None, None
    start_times = []
    end_times = []
    data_path = os.path.join(BASE_PATH, patient_id, data_type)
    try:
        for archive in archives:  # Single archive, but kept as list for consistency
            if data_type == 'cns':
                cns_path = os.path.join(data_path, archive)
                data_source = CNSDataSource(cns_path)
                for measurement in measurements:
                    try:
                        reviewer = data_source.get_modality_data_reviewer(measurement)
                        start_time = reviewer.start_time()
                        end_time = reviewer.end_time()
                        # Handle various timestamp formats
                        if isinstance(start_time, (int, float, np.integer, np.floating)):
                            # Try microseconds first
                            start_dt = datetime.fromtimestamp(float(start_time) / 1_000_000, tz=pytz.UTC)
                            # If date is implausible (e.g., before 1970 or after 2030), try nanoseconds
                            if start_dt.year < 1970 or start_dt.year > 2030:
                                start_dt = datetime.fromtimestamp(float(start_time) / 1_000_000_000, tz=pytz.UTC)
                            start_time = start_dt
                        elif isinstance(start_time, datetime):
                            if start_time.tzinfo is None:
                                start_time = start_time.replace(tzinfo=pytz.UTC)
                        else:
                            with status_output:
                                print(f"Warning: Invalid start_time type {type(start_time)} for measurement {measurement} in archive {archive}.")
                            start_time = None

                        if isinstance(end_time, (int, float, np.integer, np.floating)):
                            end_dt = datetime.fromtimestamp(float(end_time) / 1_000_000, tz=pytz.UTC)
                            if end_dt.year < 1970 or end_dt.year > 2030:
                                end_dt = datetime.fromtimestamp(float(end_time) / 1_000_000_000, tz=pytz.UTC)
                            end_time = end_dt
                        elif isinstance(end_time, datetime):
                            if end_time.tzinfo is None:
                                end_time = end_time.replace(tzinfo=pytz.UTC)
                        else:
                            with status_output:
                                print(f"Warning: Invalid end_time type {type(end_time)} for measurement {measurement} in archive {archive}.")
                            end_time = None

                        if start_time and end_time:
                            start_times.append(start_time)
                            end_times.append(end_time)
                        else:
                            with status_output:
                                print(f"Warning: No valid time range for measurement {measurement} in archive {archive}.")
                    except Exception as e:
                        with status_output:
                            print(f"Error: Failed to get time range for measurement {measurement} in archive {archive}. Details: {str(e)}")
            elif data_type == 'edf':
                raw = read_edf_data(os.path.join(data_path, archive))
                start_time = raw.info['meas_date']
                if isinstance(start_time, datetime) and start_time.tzinfo is None:
                    start_time = start_time.replace(tzinfo=pytz.UTC)
                duration = timedelta(seconds=raw.n_times / raw.info['sfreq'])
                end_time = start_time + duration
                if end_time.tzinfo is None:
                    end_time = end_time.replace(tzinfo=pytz.UTC)
                if start_time and end_time:
                    start_times.append(start_time)
                    end_times.append(end_time)
        if not start_times or not end_times:
            with status_output:
                print("Error: No valid time range found for selected measurements/archive.")
            return None, None
        min_start = min(start_times)
        max_end = max(end_times)
        with status_output:
            print(f'Available time range for selected measurements: {min_start} to {max_end}')
        return min_start, max_end
    except Exception as e:
        with status_output:
            print(f"Error: Failed to calculate time range for patient {patient_id}, data type {data_type}. Details: {str(e)}")
            traceback.print_exc()
        return None, None

def read_cns_data(cns_dir, measurement, start_time, duration_microseconds):
    try:
        data_source = CNSDataSource(cns_dir)
        reviewer = data_source.get_modality_data_reviewer(measurement)
        start_time_micro = int(start_time.timestamp() * 1_000_000)
        end_time_micro = start_time_micro + duration_microseconds
        df, start = reviewer.review_data_frame(start_time_micro, end_time_micro)
        if df.empty:
            with status_output:
                print(f"Warning: No data found for measurement {measurement} in {cns_dir} for selected time range.")
        # Ensure output is a DataFrame
        if isinstance(df, pd.Series):
            df = df.to_frame(name=measurement)
        return df
    except Exception as e:
        with status_output:
            print(f"Error: Failed to read CNS data for {measurement} in {cns_dir}. Details: {str(e)}")
            traceback.print_exc()
        return pd.DataFrame()

def read_edf_data(edf_path):
    try:
        return mne.io.read_raw_edf(edf_path, preload=True)
    except Exception as e:
        with status_output:
            print(f"Error: Failed to read EDF file {edf_path}. Ensure file is valid. Details: {str(e)}")
            traceback.print_exc()
        return None

# Export functions
def export_to_csv(df, filename):
    try:
        df.to_csv(filename)
        return filename
    except Exception as e:
        with status_output:
            print(f"Error: Failed to export CSV to {filename}. Check disk space or permissions. Details: {str(e)}")
            traceback.print_exc()
        return None

def export_to_hdf5(df, filename):
    try:
        with h5py.File(filename, 'w') as f:
            for col in df.columns:
                f.create_dataset(col, data=df[col].values)
        return filename
    except Exception as e:
        with status_output:
            print(f"Error: Failed to export HDF5 to {filename}. Check disk space or permissions. Details: {str(e)}")
            traceback.print_exc()
        return None

def export_to_edf(df, filename, sfreq, ch_names):
    try:
        n_channels = len(ch_names)
        signal = df.values.T
        f = pyedflib.EdfWriter(filename, n_channels)
        for i, ch in enumerate(ch_names):
            f.setSignalHeader(i, {'label': ch, 'dimension': 'uV', 'sample_rate': sfreq})
        f.writeSamples(signal)
        f.close()
        return filename
    except Exception as e:
        with status_output:
            print(f"Error: Failed to export EDF to {filename}. Ensure valid channel data. Details: {str(e)}")
            traceback.print_exc()
        return None

# Validation function
def validate_config(config):
    errors = []
    patient_id = config['patient']
    data_type = config['data_type']
    archives = config['archives']
    measurements = config['measurements']
    start_time = config['start_time']
    end_time = config['end_time']
    processing = config['processing']

    if not patient_id or patient_id not in get_patient_ids():
        errors.append(f"Invalid patient ID: {patient_id}. Available: {get_patient_ids()}")
    if data_type not in get_data_types(patient_id):
        errors.append(f"Invalid data type: {data_type}. Available: {get_data_types(patient_id)}")
    if not archives or len(archives) != 1:
        errors.append("Exactly one archive must be selected.")
    valid_archives = get_patient_archives(patient_id, data_type)
    for archive in archives:
        if archive not in valid_archives:
            errors.append(f"Invalid archive: {archive}. Available: {valid_archives}")
    if not measurements:
        errors.append("No measurements selected. Please select at least one measurement.")
    if data_type == 'cns':
        for archive in archives:
            valid_measurements = get_cns_measurements(os.path.join(BASE_PATH, patient_id, data_type, archive))
            for measurement in measurements:
                if measurement not in valid_measurements:
                    errors.append(f"Invalid measurement: {measurement} for archive {archive}. Available: {valid_measurements}")
    elif data_type == 'edf':
        for archive in archives:
            raw = read_edf_data(os.path.join(BASE_PATH, patient_id, data_type, archive))
            if raw:
                valid_measurements = raw.ch_names
                for measurement in measurements:
                    if measurement not in valid_measurements:
                        errors.append(f"Invalid channel: {measurement} for archive {archive}. Available: {valid_measurements}")
    min_start, max_end = get_time_range(patient_id, data_type, archives, measurements)
    if min_start and max_end:
        if start_time < min_start:
            errors.append(f"Start time {start_time} is before available data ({min_start}).")
        if end_time > max_end:
            errors.append(f"End time {end_time} is after available data ({max_end}).")
        if start_time >= end_time:
            errors.append(f"Start time {start_time} must be before end time {end_time}.")
        duration_hours = (end_time - start_time).total_seconds() / 3600
        if duration_hours > 24:
            with status_output:
                print(f"Warning: Selected time range is {duration_hours:.2f} hours, which may slow down export.")
    else:
        errors.append("Invalid time range. Check archive and measurements.")
    if 'Median Filter' in processing:
        try:
            window_size = config.get('median_window_size', 3)
            if not isinstance(window_size, int) or window_size % 2 == 0 or window_size < 1:
                errors.append("Median filter window size must be a positive odd integer (e.g., 3, 5, 7).")
        except Exception:
            errors.append("Invalid median filter window size. Use a positive odd integer (e.g., 3, 5, 7).")
    return errors

# UI Components
def safe_display(widget, fallback_message):
    try:
        if hasattr(widget, '_model_id') and widget._model_id:
            display(widget)
        else:
            with status_output:
                print(f"Warning: Widget rendering failed: {fallback_message}. Use manual export below.")
    except Exception as e:
        with status_output:
            print(f"Error displaying widget: {str(e)}. Fallback: {fallback_message}. Use manual export below.")

patient_select = widgets.Select(
    options=get_patient_ids() or ['No patients found'],
    description='Patient:',
    layout={'width': '50%'}
)

data_type_select = widgets.Select(
    options=['Select a patient first'],
    description='Data Type:',
    layout={'width': '50%'},
    disabled=True
)

archive_select = widgets.Select(
    options=['Select a data type first'],
    description='Archive:',
    layout={'width': '50%'},
    disabled=True
)

measurement_select = widgets.SelectMultiple(
    options=['Select an archive first'],
    description='Measurements:',
    layout={'width': '50%'},
    disabled=True
)

time_range_label = widgets.HTML(
    value='Available Time Range: Not yet determined'
)

time_range_start = widgets.DatetimePicker(
    description='Start Time:',
    layout={'width': '50%'},
    disabled=True
)

time_range_end = widgets.DatetimePicker(
    description='End Time:',
    layout={'width': '50%'},
    disabled=True
)

time_range_relative = widgets.Text(
    description='Hours from start (e.g., 0 to 12):',
    value='0 to 12',
    layout={'width': '50%'},
    disabled=True
)

first_6_hours_button = widgets.Button(
    description='First 6 Hours',
    disabled=True
)

first_12_hours_button = widgets.Button(
    description='First 12 Hours',
    disabled=True
)

first_24_hours_button = widgets.Button(
    description='First 24 Hours',
    disabled=True
)

all_available_button = widgets.Button(
    description='All Available',
    disabled=True
)

processing_select = widgets.SelectMultiple(
    options=['None', 'Bandpass EEG (1-40 Hz)', 'Median Filter'],
    description='Processing:',
    layout={'width': '50%'},
    disabled=True
)

median_window_input = widgets.IntText(
    value=3,
    description='Median Window Size:',
    layout={'width': '50%'},
    disabled=True
)

export_format_select = widgets.Select(
    options=['CSV', 'HDF5'],
    description='Export Format:',
    layout={'width': '50%'},
    disabled=True
)

config_name_input = widgets.Text(
    value='',
    description='Config Name:',
    layout={'width': '50%'},
    disabled=True
)

save_config_button = widgets.Button(
    description='Save Config',
    disabled=True
)

load_config_dropdown = widgets.Dropdown(
    options=[''],
    description='Load Config:',
    layout={'width': '50%'},
    disabled=True
)

preview_button = widgets.Button(
    description='Preview Data',
    disabled=True
)

export_button = widgets.Button(
    description='Export',
    disabled=True
)

stop_export_button = widgets.Button(
    description='Stop Export',
    disabled=True,
    button_style='danger'
)

clear_fields_button = widgets.Button(
    description='Clear Fields',
    disabled=False,
    button_style='warning'
)

config_upload = widgets.FileUpload(
    description='Upload Batch Config JSON',
    accept='.json',
    multiple=False,
    disabled=False
)

progress_bar = widgets.FloatProgress(
    value=0.0,
    min=0.0,
    max=1.0,
    description='Export Progress:',
    layout={'width': '50%'}
)

status_output = widgets.Output()

# Instructional content
section1_instruction = widgets.HTML(
    value='<b>1. Select Patient</b><br>Select a patient whose data you want to export.'
)

section2_instruction = widgets.HTML(
    value='<b>2. Select Data Type</b><br>Choose the type of data to export (CNS or EDF).'
)

section3_instruction = widgets.HTML(
    value='<b>3. Select Archive</b><br>Select one patient archive for the chosen data type.'
)

section4_instruction = widgets.HTML(
    value='<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
)

section5_instruction = widgets.HTML(
    value='<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
)

section6_instruction = widgets.HTML(
    value='<b>6. Select Processing</b><br>Choose signal processing (e.g., Median Filter) and set window size for Median Filter if selected.'
)

section7_instruction = widgets.HTML(
    value='<b>7. Select Export Format</b><br>Choose the file format for the exported data.'
)

section8_instruction = widgets.HTML(
    value='<b>8. Save/Load Configuration</b><br>Save your current settings or load a previous configuration.'
)

section9_instruction = widgets.HTML(
    value='<b>9. Preview and Export Data</b><br>Preview a data sample, start export, stop current export, clear fields, or upload a batch config JSON.'
)

help_section = widgets.HTML(
    value="""
    <b>Help and Troubleshooting</b><br>
    - <b>Widget errors</b>: If widgets fail to render ("model not found"), use the manual export code below.<br>
    - <b>No data found</b>: Check if patient ID, data type, archive, and measurements are valid. Run `get_patient_ids()`, `get_data_types(patient_id)`, etc., in a cell.<br>
    - <b>Export fails</b>: Ensure sufficient disk space in `exported_data/` and valid time range. Check `status_output` for details.<br>
    - <b>Slow export</b>: Large time ranges (>24 hours) or many measurements may slow exports. Use smaller ranges or monitor memory in `status_output`.<br>
    - <b>Single archive</b>: Only one archive can be selected per export. Use batch export for multiple archives.<br>
    - <b>Time range</b>: For CNS data, time range reflects only selected measurements' availability.<br>
    - <b>Median Filter</b>: Set window size (odd integer, e.g., 3, 5) when selecting Median Filter. Applies to all measurements.<br>
    - <b>Batch export</b>: Create a JSON file with a list of configs (see manual export format) and upload via "Upload Batch Config JSON".<br>
    - <b>Confirmations</b>: Each section shows the current selection (e.g., "Showing measurements for archive X").<br>
    - <b>Export Summaries</b>: Summaries and manifests are saved in `export_summaries/` with download links displayed on completion.<br>
    - <b>Contact support</b>: Share `status_output` logs and environment details (`jupyter --version`, `pip show ipywidgets`).
    """
)

# Task management
active_tasks = []
tasks_lock = Lock()
cancel_export = False
executor = ThreadPoolExecutor(max_workers=1)
current_future = None

def display_active_tasks():
    with status_output:
        if active_tasks:
            task_data = []
            current_time = time.time()
            for task in active_tasks:
                duration = int(current_time - task['start_time'])
                task_data.append({
                    'Export ID': task['export_id'],
                    'Patient': task['patient_id'],
                    'Archive': task['archive'],
                    'Measurement': task['measurement'],
                    'Start Time': datetime.fromtimestamp(task['start_time']).strftime('%Y-%m-%d %H:%M:%S'),
                    'Duration (s)': duration
                })
            df = pd.DataFrame(task_data)
            display(HTML(df.to_html(index=False, classes='table table-striped')))
        else:
            print("No active export tasks.")

# Configuration management
def save_config(_):
    try:
        config = {
            'patient': patient_select.value,
            'data_type': data_type_select.value,
            'archives': [archive_select.value] if archive_select.value else [],
            'measurements': list(measurement_select.value),
            'start_time': time_range_start.value.isoformat() if time_range_start.value else None,
            'end_time': time_range_end.value.isoformat() if time_range_end.value else None,
            'processing': list(processing_select.value),
            'export_format': export_format_select.value,
            'median_window_size': median_window_input.value if 'Median Filter' in processing_select.value else 3
        }
        config_name = config_name_input.value or f'config_{uuid.uuid4().hex[:8]}.json'
        with open(os.path.join(CONFIG_DIR, config_name), 'w') as f:
            json.dump(config, f)
        load_config_dropdown.options = [''] + [f for f in os.listdir(CONFIG_DIR) if f.endswith('.json')]
        with status_output:
            print(f'Saved configuration: {config_name}')
    except Exception as e:
        with status_output:
            print(f"Error: Failed to save configuration {config_name}. Check disk space or permissions. Details: {str(e)}")
            traceback.print_exc()

def load_config(_):
    try:
        config_file = load_config_dropdown.value
        if config_file:
            with open(os.path.join(CONFIG_DIR, config_file), 'r') as f:
                config = json.load(f)
            patient_select.value = config['patient']
            data_type_select.value = config['data_type']
            update_archives(None)
            archive_select.value = config['archives'][0] if config['archives'] else None
            update_measurements(None)
            measurement_select.value = tuple(config['measurements'])
            update_time_range(None)
            time_range_start.value = datetime.fromisoformat(config['start_time']).replace(tzinfo=pytz.UTC) if config['start_time'] else None
            time_range_end.value = datetime.fromisoformat(config['end_time']).replace(tzinfo=pytz.UTC) if config['end_time'] else None
            processing_select.value = tuple(config['processing'])
            export_format_select.value = config['export_format']
            median_window_input.value = config.get('median_window_size', 3)
            with status_output:
                print(f'Loaded configuration: {config_file}')
    except Exception as e:
        with status_output:
            print(f"Error: Failed to load configuration {config_file}. Ensure file is valid JSON. Details: {str(e)}")
            traceback.print_exc()

# Update functions
def update_processing_options(_):
    try:
        median_window_input.disabled = 'Median Filter' not in processing_select.value
        if not median_window_input.disabled:
            with status_output:
                print(f"Median Filter selected. Set window size (current: {median_window_input.value}).")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to update processing options. Details: {str(e)}")
            traceback.print_exc()

def update_data_types(_):
    try:
        if patient_select.value and patient_select.value != 'No patients found':
            patient_id = patient_select.value
            section1_instruction.value = f'<b>1. Select Patient</b><br>Patient {patient_id} selected.'
            data_type_select.options = ['Loading...']
            data_type_select.disabled = True
            safe_display(data_type_select, "Data type select widget failed to render")
            data_types = get_data_types(patient_id)
            data_type_select.options = data_types
            data_type_select.disabled = False
        else:
            section1_instruction.value = '<b>1. Select Patient</b><br>Select a patient whose data you want to export.'
            data_type_select.options = ['Select a patient first']
            data_type_select.disabled = True
        section2_instruction.value = '<b>2. Select Data Type</b><br>Choose the type of data to export (CNS or EDF).'
        section3_instruction.value = '<b>3. Select Archive</b><br>Select one patient archive for the chosen data type.'
        section4_instruction.value = '<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
        section5_instruction.value = '<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
        archive_select.options = ['Select a data type first']
        archive_select.value = None
        measurement_select.options = ['Select an archive first']
        measurement_select.value = ()
        measurement_select.disabled = True
        time_range_label.value = 'Available Time Range: Not yet determined'
        time_range_start.value = None
        time_range_end.value = None
        time_range_start.disabled = True
        time_range_end.disabled = True
        time_range_relative.value = '0 to 12'
        time_range_relative.disabled = True
        first_6_hours_button.disabled = True
        first_12_hours_button.disabled = True
        first_24_hours_button.disabled = True
        all_available_button.disabled = True
        processing_select.value = ('None',)
        processing_select.disabled = True
        median_window_input.disabled = True
        export_format_select.options = ['CSV', 'HDF5']
        export_format_select.disabled = True
        config_name_input.disabled = True
        save_config_button.disabled = True
        load_config_dropdown.disabled = True
        preview_button.disabled = True
        export_button.disabled = True
        stop_export_button.disabled = True
        safe_display(data_type_select, "Data type select widget failed to render")
        safe_display(time_range_start, "Start time picker widget failed to render")
    except Exception as e:
        data_type_select.options = ['Error loading data types']
        data_type_select.disabled = True
        with status_output:
            print(f"Error: Failed to update data types. Details: {str(e)}")
            traceback.print_exc()

def update_archives(_):
    try:
        if patient_select.value and data_type_select.value != 'none' and data_type_select.value != 'Select a patient first':
            patient_id = patient_select.value
            data_type = data_type_select.value
            section2_instruction.value = f'<b>2. Select Data Type</b><br>Data type {data_type} selected.'
            archive_select.options = ['Loading...']
            archive_select.disabled = True
            safe_display(archive_select, "Archive select widget failed to render")
            archives = get_patient_archives(patient_id, data_type)
            archive_select.options = archives or ['No archives found']
            archive_select.disabled = False
        else:
            section2_instruction.value = '<b>2. Select Data Type</b><br>Choose the type of data to export (CNS or EDF).'
            archive_select.options = ['Select a data type first']
            archive_select.value = None
            archive_select.disabled = True
        section3_instruction.value = '<b>3. Select Archive</b><br>Select one patient archive for the chosen data type.'
        section4_instruction.value = '<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
        section5_instruction.value = '<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
        measurement_select.options = ['Select an archive first']
        measurement_select.value = ()
        measurement_select.disabled = True
        time_range_label.value = 'Available Time Range: Not yet determined'
        time_range_start.value = None
        time_range_end.value = None
        time_range_start.disabled = True
        time_range_end.disabled = True
        time_range_relative.value = '0 to 12'
        time_range_relative.disabled = True
        first_6_hours_button.disabled = True
        first_12_hours_button.disabled = True
        first_24_hours_button.disabled = True
        all_available_button.disabled = True
        processing_select.value = ('None',)
        processing_select.disabled = True
        median_window_input.disabled = True
        export_format_select.options = ['CSV', 'HDF5']
        export_format_select.disabled = True
        config_name_input.disabled = True
        save_config_button.disabled = True
        load_config_dropdown.disabled = True
        preview_button.disabled = True
        export_button.disabled = True
        stop_export_button.disabled = True
        safe_display(archive_select, "Archive select widget failed to render")
        safe_display(time_range_start, "Start time picker widget failed to render")
    except Exception as e:
        archive_select.options = ['Error loading archives']
        archive_select.disabled = True
        with status_output:
            print(f"Error: Failed to update archives. Details: {str(e)}")
            traceback.print_exc()

def update_measurements(_):
    try:
        if patient_select.value and data_type_select.value != 'none' and archive_select.value and archive_select.value != 'No archives found':
            patient_id = patient_select.value
            data_type = data_type_select.value
            archive = archive_select.value
            section3_instruction.value = f'<b>3. Select Archive</b><br>Archive {archive} selected.'
            section4_instruction.value = f'<b>4. Select Measurements</b><br>Showing measurements for archive {archive}.'
            measurement_select.options = ['Loading...']
            measurement_select.disabled = True
            safe_display(measurement_select, "Measurement select widget failed to render")
            data_path = os.path.join(BASE_PATH, patient_id, data_type, archive)
            if data_type == 'cns':
                measurements = get_cns_measurements(data_path)
                measurement_select.options = measurements or ['No measurements found']
            elif data_type == 'edf':
                raw = read_edf_data(data_path)
                measurement_select.options = raw.ch_names or ['No measurements found']
            measurement_select.disabled = False
        else:
            section3_instruction.value = '<b>3. Select Archive</b><br>Select one patient archive for the chosen data type.'
            section4_instruction.value = '<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
            measurement_select.options = ['Select an archive first']
            measurement_select.value = ()
            measurement_select.disabled = True
        section5_instruction.value = '<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
        time_range_label.value = 'Available Time Range: Not yet determined'
        time_range_start.value = None
        time_range_end.value = None
        time_range_start.disabled = True
        time_range_end.disabled = True
        time_range_relative.value = '0 to 12'
        time_range_relative.disabled = True
        first_6_hours_button.disabled = True
        first_12_hours_button.disabled = True
        first_24_hours_button.disabled = True
        all_available_button.disabled = True
        processing_select.value = ('None',)
        processing_select.disabled = True
        median_window_input.disabled = True
        export_format_select.options = ['CSV', 'HDF5']
        export_format_select.disabled = True
        config_name_input.disabled = True
        save_config_button.disabled = True
        load_config_dropdown.disabled = True
        preview_button.disabled = True
        export_button.disabled = True
        stop_export_button.disabled = True
        safe_display(measurement_select, "Measurement select widget failed to render")
        safe_display(time_range_start, "Start time picker widget failed to render")
    except Exception as e:
        measurement_select.options = ['Error loading measurements']
        measurement_select.disabled = True
        with status_output:
            print(f"Error: Failed to update measurements. Details: {str(e)}")
            traceback.print_exc()

def update_time_range(_):
    try:
        with status_output:
            print('Checking time range...')
        if patient_select.value and data_type_select.value != 'none' and archive_select.value and measurement_select.value and measurement_select.value[0] != 'No measurements found':
            patient_id = patient_select.value
            data_type = data_type_select.value
            archives = [archive_select.value]
            measurements = measurement_select.value
            measurements_str = ', '.join(measurements) if measurements else 'none'
            section4_instruction.value = f'<b>4. Select Measurements</b><br>Measurements {measurements_str} selected for archive {archive_select.value}.'
            section5_instruction.value = f'<b>5. Select Time Range</b><br>Showing time range for measurements {measurements_str}.'
            start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
            with status_output:
                print(f'Calculated time range: {start_time} to {end_time}')
            if start_time and end_time and isinstance(start_time, datetime) and isinstance(end_time, datetime):
                time_range_label.value = f'Available Time Range: {start_time} to {end_time}'
                time_range_start.value = start_time
                time_range_end.value = end_time
                time_range_relative.disabled = False
                first_6_hours_button.disabled = False
                first_12_hours_button.disabled = False
                first_24_hours_button.disabled = False
                all_available_button.disabled = False
                processing_select.disabled = False
                update_processing_options(None)
                update_export_formats()
                export_format_select.disabled = False
                config_name_input.disabled = False
                save_config_button.disabled = False
                load_config_dropdown.disabled = False
                preview_button.disabled = False
                export_button.disabled = False
                stop_export_button.disabled = False
                safe_display(time_range_start, "Start time picker widget failed to render")
                safe_display(time_range_end, "End time picker widget failed to render")
            else:
                time_range_label.value = 'Available Time Range: Invalid or unavailable'
                time_range_start.value = None
                time_range_end.value = None
                time_range_start.disabled = True
                time_range_end.disabled = True
                time_range_relative.value = '0 to 12'
                time_range_relative.disabled = True
                first_6_hours_button.disabled = True
                first_12_hours_button.disabled = True
                first_24_hours_button.disabled = True
                all_available_button.disabled = True
                processing_select.value = ('None',)
                processing_select.disabled = True
                median_window_input.disabled = True
                export_format_select.options = ['CSV', 'HDF5']
                export_format_select.disabled = True
                config_name_input.disabled = True
                save_config_button.disabled = True
                load_config_dropdown.disabled = True
                preview_button.disabled = True
                export_button.disabled = True
                stop_export_button.disabled = True
                with status_output:
                    print('Error: Time range invalid. Check archive and measurements.')
        else:
            section4_instruction.value = f'<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
            section5_instruction.value = '<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
            time_range_label.value = 'Available Time Range: Not yet determined'
            time_range_start.value = None
            time_range_end.value = None
            time_range_start.disabled = True
            time_range_end.disabled = True
            time_range_relative.value = '0 to 12'
            time_range_relative.disabled = True
            first_6_hours_button.disabled = True
            first_12_hours_button.disabled = True
            first_24_hours_button.disabled = True
            all_available_button.disabled = True
            processing_select.value = ('None',)
            processing_select.disabled = True
            median_window_input.disabled = True
            export_format_select.options = ['CSV', 'HDF5']
            export_format_select.disabled = True
            config_name_input.disabled = True
            save_config_button.disabled = True
            load_config_dropdown.disabled = True
            preview_button.disabled = True
            export_button.disabled = True
            stop_export_button.disabled = True
            with status_output:
                print('Prerequisites not met for time range calculation.')
        safe_display(time_range_start, "Start time picker widget failed to render")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to update time range. Details: {str(e)}")
            traceback.print_exc()

def update_export_formats():
    try:
        if patient_select.value and data_type_select.value != 'none':
            patient_id = patient_select.value
            data_type = data_type_select.value
            data_path = os.path.join(BASE_PATH, patient_id, data_type)
            has_edf = len(glob.glob(os.path.join(data_path, '*.edf'))) > 0
            export_format_select.options = ['CSV', 'HDF5', 'EDF'] if has_edf else ['CSV', 'HDF5']
        else:
            export_format_select.options = ['CSV', 'HDF5']
    except Exception as e:
        with status_output:
            print(f"Error: Failed to update export formats. Details: {str(e)}")
            traceback.print_exc()

# Time range button handlers
def set_relative_time(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archives = [archive_select.value]
        measurements = measurement_select.value
        start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
        if start_time:
            start_hours, end_hours = map(float, time_range_relative.value.split(' to '))
            time_range_start.value = start_time + timedelta(hours=start_hours)
            time_range_end.value = start_time + timedelta(hours=end_hours)
            if time_range_end.value.tzinfo is None:
                time_range_end.value = time_range_end.value.replace(tzinfo=pytz.UTC)
            duration_hours = end_hours - start_hours
            if duration_hours > 24:
                with status_output:
                    print(f"Warning: Selected time range is {duration_hours:.2f} hours, which may slow down export.")
            safe_display(time_range_start, "Start time picker widget failed to render")
            safe_display(time_range_end, "Start time picker widget failed to render")
    except ValueError:
        with status_output:
            print("Error: Invalid relative time format. Use 'start to end' (e.g., '0 to 12').")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to set relative time range. Details: {str(e)}")
            traceback.print_exc()

def set_first_6_hours(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archives = [archive_select.value]
        measurements = measurement_select.value
        start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
        if start_time:
            time_range_start.value = start_time
            time_range_end.value = start_time + timedelta(hours=6)
            if time_range_end.value.tzinfo is None:
                time_range_end.value = time_range_end.value.replace(tzinfo=pytz.UTC)
            time_range_relative.value = '0 to 6'
            safe_display(time_range_start, "Start time picker widget failed to render")
            safe_display(time_range_end, "Start time picker widget failed to render")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to set first 6 hours. Details: {str(e)}")
            traceback.print_exc()

def set_first_12_hours(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archives = [archive_select.value]
        measurements = measurement_select.value
        start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
        if start_time:
            time_range_start.value = start_time
            time_range_end.value = start_time + timedelta(hours=12)
            if time_range_end.value.tzinfo is None:
                time_range_end.value = time_range_end.value.replace(tzinfo=pytz.UTC)
            time_range_relative.value = '0 to 12'
            safe_display(time_range_start, "Start time picker widget failed to render")
            safe_display(time_range_end, "Start time picker widget failed to render")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to set first 12 hours. Details: {str(e)}")
            traceback.print_exc()

def set_first_24_hours(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archives = [archive_select.value]
        measurements = measurement_select.value
        start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
        if start_time:
            time_range_start.value = start_time
            time_range_end.value = start_time + timedelta(hours=24)
            if time_range_end.value.tzinfo is None:
                time_range_end.value = time_range_end.value.replace(tzinfo=pytz.UTC)
            time_range_relative.value = '0 to 24'
            with status_output:
                print("Warning: 24-hour range may slow down export.")
            safe_display(time_range_start, "Start time picker widget failed to render")
            safe_display(time_range_end, "Start time picker widget failed to render")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to set first 24 hours. Details: {str(e)}")
            traceback.print_exc()

def set_all_available(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archives = [archive_select.value]
        measurements = measurement_select.value
        start_time, end_time = get_time_range(patient_id, data_type, archives, measurements)
        if start_time and end_time:
            time_range_start.value = start_time
            time_range_end.value = end_time
            duration_hours = (end_time - start_time).total_seconds() / 3600
            time_range_relative.value = f'0 to {duration_hours}'
            if duration_hours > 24:
                with status_output:
                    print(f"Warning: Full range is {duration_hours:.2f} hours, which may slow down export.")
            safe_display(time_range_start, "Start time picker widget failed to render")
            safe_display(time_range_end, "Start time picker widget failed to render")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to set all available time range. Details: {str(e)}")
            traceback.print_exc()

# Preview function
def preview_data(_):
    try:
        patient_id = patient_select.value
        data_type = data_type_select.value
        archive = archive_select.value
        measurements = measurement_select.value
        start_time = time_range_start.value
        data_path = os.path.join(BASE_PATH, patient_id, data_type, archive)
        with status_output:
            print(f"Generating previews for {patient_id}, {data_type}, {archive}, measurements: {', '.join(measurements)}...")
        for measurement in measurements:
            with status_output:
                print(f"Previewing measurement: {measurement}")
            if data_type == 'cns':
                df = read_cns_data(data_path, measurement, start_time, duration_microseconds=60_000_000)  # 60s sample
                sfreq = 1 / (df.index[1] - df.index[0]).total_seconds() if len(df) > 1 else 1.0
            elif data_type == 'edf':
                raw = read_edf_data(data_path)
                sfreq = raw.info['sfreq']
                start_idx = int((start_time - raw.info['meas_date']).total_seconds() * sfreq)
                end_idx = start_idx + int(60 * sfreq)  # 60s sample
                data, _ = raw[measurement, start_idx:end_idx]
                # Ensure data is 2D for single channel
                if data.ndim == 1:
                    data = data.reshape(1, -1)
                df = pd.DataFrame(data.T, columns=[measurement])
            with status_output:
                if not df.empty:
                    print(f"Data type: {type(df)}")
                    # Convert Series to DataFrame if necessary
                    if isinstance(df, pd.Series):
                        df = df.to_frame(name=measurement)
                    print(f"Measurement: {measurement}, Sampling rate: {sfreq:.2f} Hz, Samples: {len(df)}, Missing values: {df.isna().sum().sum()}")
                    display(HTML(df.head(100).to_html(classes='table table-striped')))
                else:
                    print(f"Warning: No data available for preview of measurement {measurement}. Check time range or measurement.")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to generate preview for measurement {measurement}. Details: {str(e)}")
            traceback.print_exc()

# Export function
def perform_export(config):
    global cancel_export
    export_id = config['export_id']
    manifest = {'export_id': export_id, 'files': [], 'errors': []}
    export_start_time = time.time()
    median_window_size = config.get('median_window_size', 3)
    # Print to both status_output and stdout
    with status_output:
        print(f"Starting export {export_id}...")
    print(f"Starting export {export_id}...")  # stdout fallback
    try:
        # Debug config['measurements']
        with status_output:
            print(f"DEBUG: config['measurements'] = {config['measurements']}")
        print(f"DEBUG: config['measurements'] = {config['measurements']}")  # stdout
        total_steps = len(config['measurements'])
        with status_output:
            print(f"DEBUG: total_steps = {total_steps}")
        print(f"DEBUG: total_steps = {total_steps}")  # stdout
        current_step = 0
        progress_bar.value = 0.0
        # Ensure widget update is synchronized
        with status_output:
            print(f'Export ID: {export_id} ({total_steps} total steps)')
            display(status_output)  # Force widget refresh
        print(f'Export ID: {export_id} ({total_steps} total steps)')  # stdout fallback
        patient_id = config['patient']
        archive = config['archives'][0]  # Single archive
        data_path = os.path.join(BASE_PATH, patient_id, config['data_type'], archive)
        task_start_time = time.time()
        with status_output:
            print(f'Processing patient={patient_id}, archive={archive}')
        print(f'Processing patient={patient_id}, archive={archive}')  # stdout
        if config['data_type'] == 'cns':
            data_source = CNSDataSource(data_path)
            start_time = config['start_time']
            duration_microseconds = int((config['end_time'] - config['start_time']).total_seconds() * 1_000_000)
            for measurement in config['measurements']:
                if cancel_export:
                    with status_output:
                        print(f'Export cancelled: {export_id}')
                    print(f'Export cancelled: {export_id}')  # stdout
                    manifest['errors'].append('Export cancelled by user')
                    break
                if time.time() - task_start_time > EXPORT_TIMEOUT:
                    with status_output:
                        print(f"Timeout after {EXPORT_TIMEOUT}s for {measurement}. Saving partial results.")
                    print(f"Timeout after {EXPORT_TIMEOUT}s for {measurement}.")  # stdout
                    manifest['errors'].append(f"Timeout for {measurement}")
                    break
                mem = psutil.Process().memory_info().rss / 1024**2
                with status_output:
                    print(f'Exporting measurement={measurement}, memory: {mem:.2f}MB')
                print(f'Exporting measurement={measurement}, memory: {mem:.2f}MB')  # stdout
                with tasks_lock:
                    active_tasks.append({
                        'export_id': export_id,
                        'patient_id': patient_id,
                        'archive': archive,
                        'measurement': measurement,
                        'start_time': time.time()
                    })
                with status_output:
                    display_active_tasks()
                df = read_cns_data(data_path, measurement, start_time, duration_microseconds)
                sfreq = 1 / (df.index[1] - df.index[0]).total_seconds() if len(df) > 1 else 1.0
                for proc in config['processing']:
                    if proc == 'Bandpass EEG (1-40 Hz)' and 'EEG' in measurement:
                        for col in df.columns:
                            df[col] = bandpass_filter(df[col], 1, 40, sfreq)
                    elif proc == 'Median Filter':
                        for col in df.columns:
                            df[col] = median_filter(df[col], kernel_size=median_window_size)
                filename = f'{EXPORT_DIR}/{patient_id}_{archive}_{measurement}_{uuid.uuid4().hex[:8]}.{config["export_format"].lower()}'
                if os.path.exists(filename):
                    with status_output:
                        print(f"Skipping existing file: {filename}")
                    print(f"Skipping existing file: {filename}")  # stdout
                    manifest['files'].append(filename)
                    continue
                if config['export_format'] == 'CSV':
                    filename = export_to_csv(df, filename)
                elif config['export_format'] == 'HDF5':
                    filename = export_to_hdf5(df, filename)
                elif config['export_format'] == 'EDF':
                    filename = export_to_edf(df, filename, sfreq, df.columns)
                if filename:
                    with status_output:
                        print(f'Exported to {filename}')
                        display(FileLink(filename))  # Display download link for each file
                    print(f'Exported to {filename}')  # stdout
                    manifest['files'].append(filename)
                else:
                    manifest['errors'].append(f"Failed to export {measurement} to {filename}")
                with tasks_lock:
                    active_tasks[:] = [t for t in active_tasks if not (t['export_id'] == export_id and t['measurement'] == measurement)]
                current_step += 1
                progress_bar.value = current_step / total_steps
                with status_output:
                    safe_display(progress_bar, f"Progress: {current_step}/{total_steps}")
                    display_active_tasks()
        elif config['data_type'] == 'edf':
            if time.time() - task_start_time > EXPORT_TIMEOUT:
                with status_output:
                    print(f"Timeout after {EXPORT_TIMEOUT}s for {archive}. Saving partial results.")
                print(f"Timeout after {EXPORT_TIMEOUT}s for {archive}.")  # stdout
                manifest['errors'].append(f"Timeout for {archive}")
            else:
                raw = read_edf_data(data_path)
                sfreq = raw.info['sfreq']
                start_idx = int((config['start_time'] - raw.info['meas_date']).total_seconds() * sfreq)
                end_idx = int((config['end_time'] - raw.info['meas_date']).total_seconds() * sfreq)
                selected_channels = [ch for ch in config['measurements'] if ch in raw.ch_names]
                if not selected_channels:
                    with status_output:
                        print(f'Error: No valid channels found for measurements={config["measurements"]}. Available: {raw.ch_names}')
                    print(f'Error: No valid channels found for measurements={config["measurements"]}.')  # stdout
                    manifest['errors'].append(f"No valid channels for {archive}")
                else:
                    mem = psutil.Process().memory_info().rss / 1024**2
                    with status_output:
                        print(f'Exporting channels={selected_channels}, memory: {mem:.2f}MB')
                    print(f'Exporting channels={selected_channels}, memory: {mem:.2f}MB')  # stdout
                    with tasks_lock:
                        active_tasks.append({
                            'export_id': export_id,
                            'patient_id': patient_id,
                            'archive': archive,
                            'measurement': ', '.join(selected_channels),
                            'start_time': time.time()
                        })
                    with status_output:
                        display_active_tasks()
                    raw = raw.pick_channels(selected_channels)
                    data, times = raw[:, start_idx:end_idx]
                    df = pd.DataFrame(data.T, columns=selected_channels)
                    for proc in config['processing']:
                        if proc == 'Bandpass EEG (1-40 Hz)':
                            for col in df.columns:
                                df[col] = bandpass_filter(df[col], 1, 40, sfreq)
                        elif proc == 'Median Filter':
                            for col in df.columns:
                                df[col] = median_filter(df[col], kernel_size=median_window_size)
                    filename = f'{EXPORT_DIR}/{patient_id}_{archive}_edf_{uuid.uuid4().hex[:8]}.{config["export_format"].lower()}'
                    if os.path.exists(filename):
                        with status_output:
                            print(f"Skipping existing file: {filename}")
                        print(f"Skipping existing file: {filename}")  # stdout
                        manifest['files'].append(filename)
                    else:
                        if config['export_format'] == 'CSV':
                            filename = export_to_csv(df, filename)
                        elif config['export_format'] == 'HDF5':
                            filename = export_to_hdf5(df, filename)
                        elif config['export_format'] == 'EDF':
                            filename = export_to_edf(df, filename, sfreq, selected_channels)
                        if filename:
                            with status_output:
                                print(f'Exported to {filename}')
                                display(FileLink(filename))  # Display download link for each file
                            print(f'Exported to {filename}')  # stdout
                            manifest['files'].append(filename)
                        else:
                            manifest['errors'].append(f"Failed to export channels {selected_channels} to {filename}")
                    with tasks_lock:
                        active_tasks[:] = [t for t in active_tasks if not (t['export_id'] == export_id and t['measurement'] == ', '.join(selected_channels))]
                    current_step += 1
                    progress_bar.value = current_step / total_steps
                    with status_output:
                        safe_display(progress_bar, f"Progress: {current_step}/{total_steps}")
                        display_active_tasks()
        duration = time.time() - export_start_time
        total_size = sum(os.path.getsize(f) for f in manifest['files'] if os.path.exists(f)) / 1024**2
        summary = f"""
Export Summary: {export_id}
- Files exported: {len(manifest['files'])}
- Total size: {total_size:.2f} MB
- Duration: {duration:.2f} seconds
- Processing applied: {', '.join(config['processing'])}
- Median window size: {median_window_size if 'Median Filter' in config['processing'] else 'N/A'}
- Errors: {len(manifest['errors'])}
"""
        if manifest['errors']:
            summary += "Errors encountered:\n" + '\n'.join(f"- {e}" for e in manifest['errors'])
        summary_filename = f'{EXPORT_SUMMARIES_DIR}/export_summary_{export_id}.txt'
        with open(summary_filename, 'w') as f:
            f.write(summary)
        manifest_filename = f'{EXPORT_SUMMARIES_DIR}/export_manifest_{export_id}.json'
        with open(manifest_filename, 'w') as f:
            json.dump(manifest, f)
        with status_output:
            print(f"<b>Export {export_id} Completed Successfully</b>")
            print(summary)
            print("Download exported files:")
            for file in manifest['files']:
                display(FileLink(file))
            print(f"Download summary: {summary_filename}")
            display(FileLink(summary_filename))
            display(status_output)  # Force widget refresh
        print(f"Export {export_id} Completed Successfully")  # stdout
        print(summary)  # stdout
        print(f"Summary saved to: {summary_filename}")  # stdout
        if not cancel_export:
            with status_output:
                print(f'Completed export: {export_id} ({current_step}/{total_steps} steps)')
                display_active_tasks()
            print(f'Completed export: {export_id} ({current_step}/{total_steps} steps)')  # stdout
        else:
            with status_output:
                print(f'Export cancelled: {export_id} ({current_step}/{total_steps} steps)')
                display_active_tasks()
            print(f'Export cancelled: {export_id} ({current_step}/{total_steps} steps)')  # stdout
    except Exception as e:
        with status_output:
            print(f"Error: Failed to complete export {export_id}. Details: {str(e)}")
            traceback.print_exc()
            manifest['errors'].append(f"Unexpected error: {str(e)}")
            display_active_tasks()
        print(f"Error: Failed to complete export {export_id}. Details: {str(e)}")  # stdout
        # Save partial manifest and summary even on error
        summary_filename = f'{EXPORT_SUMMARIES_DIR}/export_summary_{export_id}.txt'
        with open(summary_filename, 'w') as f:
            f.write(f"Export {export_id} Failed\nError: {str(e)}\nFiles exported: {len(manifest['files'])}\n")
        manifest_filename = f'{EXPORT_SUMMARIES_DIR}/export_manifest_{export_id}.json'
        with open(manifest_filename, 'w') as f:
            json.dump(manifest, f)
        with status_output:
            print(f"Download partial summary: {summary_filename}")
            display(FileLink(summary_filename))
    progress_bar.value = 0.0
    with status_output:
        safe_display(progress_bar, "Progress bar reset to 0")
    with tasks_lock:
        active_tasks[:] = [t for t in active_tasks if t['export_id'] != export_id]

def start_export(_):
    global cancel_export, current_future
    try:
        with status_output:
            print("Export button clicked")
        if current_future and not current_future.done():
            with status_output:
                print("Error: An export is already in progress. Stop the current export or wait for completion.")
            return
        config = {
            'export_id': uuid.uuid4().hex,
            'patient': patient_select.value,
            'data_type': data_type_select.value,
            'archives': [archive_select.value] if archive_select.value else [],
            'measurements': list(measurement_select.value),
            'start_time': time_range_start.value,
            'end_time': time_range_end.value,
            'processing': list(processing_select.value),
            'export_format': export_format_select.value,
            'median_window_size': median_window_input.value if 'Median Filter' in processing_select.value else 3
        }
        errors = validate_config(config)
        if errors:
            with status_output:
                print("Error: Cannot start export due to invalid configuration:")
                for error in errors:
                    print(f"- {error}")
            return
        with status_output:
            print(f'Submitting export: {config["export_id"]}')
            print(f'Configuration: {config}')
        cancel_export = False
        current_future = executor.submit(perform_export, config)
        with status_output:
            print("Export submitted successfully")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to start export. Details: {str(e)}")
            traceback.print_exc()

def stop_export(_):
    global cancel_export, active_tasks, current_future
    try:
        with status_output:
            print("Stop export button clicked")
        with tasks_lock:
            cancel_export = True
            active_tasks.clear()
        if current_future and not current_future.done():
            current_future.cancel()
        with status_output:
            print("Export stopped and tasks cleared.")
            display_active_tasks()
        progress_bar.value = 0.0
        safe_display(progress_bar, "Progress bar reset to 0")
        cancel_export = False
        current_future = None
    except Exception as e:
        with status_output:
            print(f"Error: Failed to stop export. Details: {str(e)}")
            traceback.print_exc()

def clear_fields(_):
    try:
        with status_output:
            print("Clear fields button clicked")
        patient_select.value = None
        section1_instruction.value = '<b>1. Select Patient</b><br>Select a patient whose data you want to export.'
        data_type_select.options = ['Select a patient first']
        data_type_select.value = 'Select a patient first'
        data_type_select.disabled = True
        section2_instruction.value = '<b>2. Select Data Type</b><br>Choose the type of data to export (CNS or EDF).'
        archive_select.options = ['Select a data type first']
        archive_select.value = None
        archive_select.disabled = True
        section3_instruction.value = '<b>3. Select Archive</b><br>Select one patient archive for the chosen data type.'
        measurement_select.options = ['Select an archive first']
        measurement_select.value = ()
        measurement_select.disabled = True
        section4_instruction.value = '<b>4. Select Measurements</b><br>Select the specific measurements or channels to export.'
        time_range_label.value = 'Available Time Range: Not yet determined'
        time_range_start.value = None
        time_range_end.value = None
        time_range_start.disabled = True
        time_range_end.disabled = True
        time_range_relative.value = '0 to 12'
        time_range_relative.disabled = True
        section5_instruction.value = '<b>5. Select Time Range</b><br>Choose start/end times or hours from data start (e.g., "0 to 12"). Use buttons for common ranges.'
        first_6_hours_button.disabled = True
        first_12_hours_button.disabled = True
        first_24_hours_button.disabled = True
        all_available_button.disabled = True
        processing_select.value = ('None',)
        processing_select.disabled = True
        median_window_input.value = 3
        median_window_input.disabled = True
        export_format_select.options = ['CSV', 'HDF5']
        export_format_select.value = 'CSV'
        export_format_select.disabled = True
        config_name_input.value = ''
        config_name_input.disabled = True
        save_config_button.disabled = True
        load_config_dropdown.value = ''
        load_config_dropdown.disabled = True
        preview_button.disabled = True
        export_button.disabled = True
        stop_export_button.disabled = True
        progress_bar.value = 0.0
        with status_output:
            print("All fields cleared.")
            safe_display(progress_bar, "Progress bar reset to 0")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to clear fields. Details: {str(e)}")
            traceback.print_exc()

def process_batch_config(_):
    global cancel_export, current_future
    try:
        if not config_upload.value:
            with status_output:
                print("Error: No batch config file uploaded.")
            return
        config_data = json.loads(list(config_upload.value.values())[0]['content'].decode())
        if not isinstance(config_data, list):
            with status_output:
                print("Error: Batch config JSON must be a list of configurations.")
            return
        with status_output:
            print(f"Processing batch export with {len(config_data)} configurations...")
        for i, config in enumerate(config_data, 1):
            if current_future and not current_future.done():
                with status_output:
                    print(f"Waiting for previous export to complete before config {i}/{len(config_data)}...")
                current_future.result()  # Wait for completion
            config['export_id'] = uuid.uuid4().hex
            config['median_window_size'] = config.get('median_window_size', 3)
            errors = validate_config(config)
            if errors:
                with status_output:
                    print(f"Error in batch config {i}:")
                    for error in errors:
                        print(f"- {error}")
                continue
            with status_output:
                print(f'Submitting batch config {i}/{len(config_data)}: {config["export_id"]}')
            cancel_export = False
            current_future = executor.submit(perform_export, config)
            current_future.result()  # Wait for this export to complete
        with status_output:
            print("Batch export completed.")
    except Exception as e:
        with status_output:
            print(f"Error: Failed to process batch config. Ensure JSON format is correct. Details: {str(e)}")
            traceback.print_exc()

# Event handlers
patient_select.observe(update_data_types, names='value')
data_type_select.observe(update_archives, names='value')
archive_select.observe(update_measurements, names='value')
measurement_select.observe(update_time_range, names='value')
time_range_relative.observe(set_relative_time, names='value')
first_6_hours_button.on_click(set_first_6_hours)
first_12_hours_button.on_click(set_first_12_hours)
first_24_hours_button.on_click(set_first_24_hours)
all_available_button.on_click(set_all_available)
processing_select.observe(update_processing_options, names='value')
save_config_button.on_click(save_config)
load_config_dropdown.observe(load_config, names='value')
preview_button.on_click(preview_data)
export_button.on_click(start_export)
stop_export_button.on_click(stop_export)
clear_fields_button.on_click(clear_fields)
config_upload.observe(process_batch_config, names='value')

# Display UI
check_widgets_environment()
try:
    display(widgets.VBox([
        help_section,
        section1_instruction,
        patient_select,
        section2_instruction,
        data_type_select,
        section3_instruction,
        archive_select,
        section4_instruction,
        measurement_select,
        section5_instruction,
        time_range_label,
        time_range_start,
        time_range_end,
        time_range_relative,
        widgets.HBox([first_6_hours_button, first_12_hours_button, first_24_hours_button, all_available_button]),
        section6_instruction,
        processing_select,
        median_window_input,
        section7_instruction,
        export_format_select,
        section8_instruction,
        config_name_input,
        save_config_button,
        load_config_dropdown,
        section9_instruction,
        widgets.HBox([preview_button, export_button, stop_export_button, clear_fields_button]),
        config_upload,
        progress_bar,
        status_output
    ]))
except Exception as e:
    with status_output:
        print(f"Error: Failed to display UI. Use manual export below. Details: {str(e)}")
        traceback.print_exc()

# Initialize config dropdown
try:
    load_config_dropdown.options = [''] + [f for f in os.listdir(CONFIG_DIR) if f.endswith('.json')]
except Exception as e:
    with status_output:
        print(f"Error: Failed to load saved configurations. Check {CONFIG_DIR}. Details: {str(e)}")
        traceback.print_exc()

ipywidgets version: 8.0.2
widgetsnbextension is not installed. Run: jupyter nbextension enable --py widgetsnbextension --sys-prefix
